<a href="https://colab.research.google.com/github/MarigoldJ/ygl2/blob/main/class/20210618_nlp_day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>